Trabalho Big Data

Gabriel Rios de Carvalho - Matrícula 2014000031

Professor: Paulo Alves Braz​

Trabalho realizado utilizando o PySpark para consultas SQL.

Base de dados disponível em: https://dados.gov.br/dados/conjuntos-dados/imoveis-da-uniao

# Instalação do Pyspark

In [ ]:
pip install pyspark

# Configuração de sessão do Pyspark e importação da base de dados

In [3]:
from pyspark.sql import SparkSession
import pandas as pd

#criar sessão spark
spark = SparkSession.builder.master('local[*]').getOrCreate()

# download da base de dados
!wget --quiet --show-progress https://www.gov.br/economia/pt-br/assuntos/patrimonio-da-uniao/transparencia/dados-abertos/arquivos/2022/da-imoveisdauniao-_20220815.xlsx

# carregando base de dados
df_spark = pd.read_excel("./da-imoveisdauniao-_20220815.xlsx")

# criando dataframe spark
df = spark.createDataFrame(df_spark)


da-imoveisdauniao-_   5%[>                   ]   3.25M   733KB/s    in 5.0s    
da-imoveisdauniao-_ 100%[+==================>]  54.67M   732KB/s    in 74s     


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# Exibindo algumas informações do Dataframe

In [6]:
# Informações sobre os tipos de dados das colunas
df.printSchema()
# Exibir primeiras linhas do dataframe
df.show(10)
# Retornar quantidade total de instâncias do dataframe
df.count()

root
 |-- Classe: string (nullable = true)
 |-- Número do RIP: long (nullable = true)
 |-- Id. Utilização: double (nullable = true)
 |-- Data de cadastramento: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Endereço: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Conceituação: string (nullable = true)
 |-- Tipo Imóvel: string (nullable = true)
 |-- Regime de Utilização: string (nullable = true)
 |-- Proprietário Oficial: string (nullable = true)
 |-- Data do Início da Utilização: string (nullable = true)
 |-- Área do Terreno Total: string (nullable = true)
 |-- Área da União: string (nullable = true)

+--------+-------------+--------------+---------------------+---+-------------+--------------------+---------+--------------------+-----------+--------------------+--------------------+----------------------------+---------------------+---------------+
|  Classe|Número do RIP|Id. Utilização|Data de cadastrament

745106

# Filtragem dos dados

In [7]:
# Seleção das colunas a serem utilizadas no trabalho
df_filtered = df.select('Número do RIP','Classe', 'UF', 'Município', 'Conceituação','Tipo Imóvel')
df_filtered.show(10)

# Informações sobre os tipos de dados das colunas
df_filtered.printSchema()

# Retornar quantidade total de instâncias do dataframe
df_filtered.count()

+-------------+--------+---+-------------+--------------------+-----------+
|Número do RIP|  Classe| UF|    Município|        Conceituação|Tipo Imóvel|
+-------------+--------+---+-------------+--------------------+-----------+
|   1010000141|DOMINIAL| RO|GUAJARA-MIRIM|     MARGINAL DE RIO|    TERRENO|
|   1010000222|DOMINIAL| RO|GUAJARA-MIRIM|     MARGINAL DE RIO|    TERRENO|
|   1010000303|DOMINIAL| RO|GUAJARA-MIRIM|     MARGINAL DE RIO|    TERRENO|
|   1010000494|DOMINIAL| RO|GUAJARA-MIRIM|     MARGINAL DE RIO|    TERRENO|
|   1010000737|DOMINIAL| RO|GUAJARA-MIRIM|MARGINAL DE RIO C...|    TERRENO|
|   1010000818|DOMINIAL| RO|GUAJARA-MIRIM|   NACIONAL INTERIOR|    TERRENO|
|   1010000907|DOMINIAL| RO|GUAJARA-MIRIM|MARGINAL DE RIO C...|    TERRENO|
|   1010001032|DOMINIAL| RO|GUAJARA-MIRIM|MARGINAL DE RIO C...|    TERRENO|
|   1010001113|DOMINIAL| RO|GUAJARA-MIRIM|   NACIONAL INTERIOR|    TERRENO|
|   1010001202|DOMINIAL| RO|GUAJARA-MIRIM|   NACIONAL INTERIOR|    TERRENO|
+-----------

745106

In [8]:
df_filtered=df_filtered.withColumnRenamed("Município", "Municipio").withColumnRenamed("Conceituação", "Conceituacao").withColumnRenamed("Tipo Imóvel", "TipoImovel")
df_filtered.show()


+-------------+--------+---+-------------+--------------------+----------+
|Número do RIP|  Classe| UF|    Municipio|        Conceituacao|TipoImovel|
+-------------+--------+---+-------------+--------------------+----------+
|   1010000141|DOMINIAL| RO|GUAJARA-MIRIM|     MARGINAL DE RIO|   TERRENO|
|   1010000222|DOMINIAL| RO|GUAJARA-MIRIM|     MARGINAL DE RIO|   TERRENO|
|   1010000303|DOMINIAL| RO|GUAJARA-MIRIM|     MARGINAL DE RIO|   TERRENO|
|   1010000494|DOMINIAL| RO|GUAJARA-MIRIM|     MARGINAL DE RIO|   TERRENO|
|   1010000737|DOMINIAL| RO|GUAJARA-MIRIM|MARGINAL DE RIO C...|   TERRENO|
|   1010000818|DOMINIAL| RO|GUAJARA-MIRIM|   NACIONAL INTERIOR|   TERRENO|
|   1010000907|DOMINIAL| RO|GUAJARA-MIRIM|MARGINAL DE RIO C...|   TERRENO|
|   1010001032|DOMINIAL| RO|GUAJARA-MIRIM|MARGINAL DE RIO C...|   TERRENO|
|   1010001113|DOMINIAL| RO|GUAJARA-MIRIM|   NACIONAL INTERIOR|   TERRENO|
|   1010001202|DOMINIAL| RO|GUAJARA-MIRIM|   NACIONAL INTERIOR|   TERRENO|
|   1010001385|DOMINIAL| 

# Exportando df em formato Parquet

In [9]:
df_filtered.write.format('parquet').mode('overwrite').save('imoveisdauniao_filtered_parquet')


# Lendo arquivo em formato Parquet

In [10]:
read_df_parquet = spark.read.format('parquet').load('imoveisdauniao_filtered_parquet')

In [11]:
read_df_parquet.show(10)

+-------------+--------+---+---------+------------+----------+
|Número do RIP|  Classe| UF|Municipio|Conceituacao|TipoImovel|
+-------------+--------+---+---------+------------+----------+
|5705010995415|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010995504|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010995687|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010995768|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010995849|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010995920|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010996063|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010996144|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010996225|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
|5705010996306|DOMINIAL| ES|  VITORIA|     MARINHA|   TERRENO|
+-------------+--------+---+---------+------------+----------+
only showing top 10 rows



#Consultas SQL

In [12]:
# Habilitando Dataframe para consultas SQL
read_df_parquet.createOrReplaceTempView('imoveisUniao')

In [13]:
# Imóveis Funcionais no DF
consulta = spark.sql(" \
            SELECT * \
            FROM imoveisUniao \
            WHERE UF = 'DF' and Classe = 'IMOVEL FUNCIONAL' \
            ORDER BY TipoImovel")

consulta.show(5)
consulta.count()

+-------------+----------------+---+---------+-----------------+-----------+
|Número do RIP|          Classe| UF|Municipio|     Conceituacao| TipoImovel|
+-------------+----------------+---+---------+-----------------+-----------+
|9701309715000|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|  ALFÂNDEGA|
|9701159075000|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|APARTAMENTO|
|9701159235008|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|APARTAMENTO|
|9701159255009|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|APARTAMENTO|
|9701159265004|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|APARTAMENTO|
+-------------+----------------+---+---------+-----------------+-----------+
only showing top 5 rows



7162

In [14]:
# Imóveis Funcionais Totais
consulta = spark.sql(" \
            SELECT * \
            FROM imoveisUniao \
            WHERE Classe = 'IMOVEL FUNCIONAL' \
            ORDER BY TipoImovel")

consulta.show(5)
consulta.count()

+-------------+----------------+---+---------+-----------------+-----------+
|Número do RIP|          Classe| UF|Municipio|     Conceituacao| TipoImovel|
+-------------+----------------+---+---------+-----------------+-----------+
|9701309715000|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|  ALFÂNDEGA|
|9701159275000|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|APARTAMENTO|
|9701159615005|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|APARTAMENTO|
|9701159285005|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|APARTAMENTO|
|9701159075000|IMOVEL FUNCIONAL| DF| BRASILIA|NACIONAL INTERIOR|APARTAMENTO|
+-------------+----------------+---+---------+-----------------+-----------+
only showing top 5 rows



7292

In [15]:
# Conceituação: Terras Indígenas (Total)

consulta = spark.sql(" \
            SELECT * \
            FROM imoveisUniao \
            WHERE Conceituacao in ('TERRA INDÍGENA', 'TERRA INDIGENA') \
            ORDER BY UF")

consulta.show(5)
consulta.count()

+-------------+------------+---+---------+--------------+----------+
|Número do RIP|      Classe| UF|Municipio|  Conceituacao|TipoImovel|
+-------------+------------+---+---------+--------------+----------+
| 113000175003|USO ESPECIAL| AC|    FEIJO|TERRA INDÍGENA|   RESERVA|
| 113000355001|USO ESPECIAL| AC|    FEIJO|TERRA INDÍGENA|   RESERVA|
| 113000195004|USO ESPECIAL| AC|    FEIJO|TERRA INDÍGENA|   RESERVA|
| 113000145007|USO ESPECIAL| AC|    FEIJO|TERRA INDÍGENA|   RESERVA|
| 113000205000|USO ESPECIAL| AC|    FEIJO|TERRA INDÍGENA|   RESERVA|
+-------------+------------+---+---------+--------------+----------+
only showing top 5 rows



716

In [16]:
# Conceituação: Terras Indígenas (Norte)

consulta = spark.sql(" \
            SELECT * \
            FROM imoveisUniao \
            WHERE Conceituacao in ('TERRA INDÍGENA', 'TERRA INDIGENA') \
            and UF in ('AC', 'AP' , 'AM', 'PA', 'RO', 'RR', 'TO')\
            ORDER BY UF")

consulta.show(5)
consulta.count()

+-------------+------------+---+---------------+--------------+----------+
|Número do RIP|      Classe| UF|      Municipio|  Conceituacao|TipoImovel|
+-------------+------------+---+---------------+--------------+----------+
| 107000555006|USO ESPECIAL| AC|CRUZEIRO DO SUL|TERRA INDÍGENA|   RESERVA|
| 113000195004|USO ESPECIAL| AC|          FEIJO|TERRA INDÍGENA|   RESERVA|
| 109000025003|USO ESPECIAL| AC|    MANCIO LIMA|TERRA INDÍGENA|   RESERVA|
| 109000035009|USO ESPECIAL| AC|    MANCIO LIMA|TERRA INDÍGENA|   RESERVA|
| 113000145007|USO ESPECIAL| AC|          FEIJO|TERRA INDÍGENA|   RESERVA|
+-------------+------------+---+---------------+--------------+----------+
only showing top 5 rows



407

In [17]:
# Conceituação: Terras Indígenas (Centro-Sul + Nordeste)

consulta = spark.sql(" \
            SELECT * \
            FROM imoveisUniao \
            WHERE Conceituacao in ('TERRA INDÍGENA', 'TERRA INDIGENA') \
            and UF in ('RJ','SP','MG','ES', 'PR', 'RS', 'SC', 'DF', 'GO', \
            'MT', 'MS','AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE')\
            ORDER BY UF")

consulta.show(5)
consulta.count()

+-------------+------------+---+------------+--------------+----------+
|Número do RIP|      Classe| UF|   Municipio|  Conceituacao|TipoImovel|
+-------------+------------+---+------------+--------------+----------+
|2645000095000|USO ESPECIAL| AL|   PARICONHA|TERRA INDÍGENA|      CASA|
|2729000035002|USO ESPECIAL| AL|CAMPO GRANDE|TERRA INDÍGENA|   RESERVA|
|2645000115001|USO ESPECIAL| AL|   PARICONHA|TERRA INDÍGENA|    OUTROS|
|2645000135002|USO ESPECIAL| AL|   PARICONHA|TERRA INDÍGENA|    OUTROS|
|2701000175009|USO ESPECIAL| AL| AGUA BRANCA|TERRA INDÍGENA|   RESERVA|
+-------------+------------+---+------------+--------------+----------+
only showing top 5 rows



309